# 0L Contributors payments report 
## Generate report
### Pre-requisites
- access to well formed payments.json files
- a correctly configured config.py file
### TODO

### Imports and loading variables

In [10]:
import requests
import json
import pandas as pd
import os

from config import Config

f = open(Config.CONTRIBUTORS_FILEPATH)
contributors = json.load(f)
print(contributors)

{'5F8AC83A9B3BF2EFF20A6C16CD05C111': {'identities': [{'context': 'Discord', 'id': '776520842070720514', 'username': 'Paulito#0564'}, {'context': 'Github', 'id': 'paulito123', 'username': 'Paulito'}], 'roles': [{'name': 'broker', 'context': 'analytics-wg'}]}, '5DB97DD068263449C8EB369DC2312AFA': {'identities': [{'context': 'Discord', 'id': '776520842070799999', 'username': 'Paulito#9999'}, {'context': 'Github', 'id': 'paulito999', 'username': 'Dummito'}], 'roles': [{'name': 'developer', 'context': 'analytics-wg'}]}, 'E8DF1AFD258A10BD0392E1A8D274CC81': {'identities': [{'context': 'Discord', 'id': '776520842070700000', 'username': 'Paulito#0000'}, {'context': 'Github', 'id': 'paulito000', 'username': 'Dummito2'}], 'roles': [{'name': 'developer', 'context': 'hustle-karma-wg'}]}}


### Flatten data

In [11]:
# assign directory
directory = Config.PAYMENTS_DIR

# define output
output = []

# flatten payment files
for filename in os.listdir(directory):
    fq_filename = os.path.join(directory, filename)

    if os.path.isfile(fq_filename) and filename[-5:] == ".json":
        # print(fq_filename)
        with open(fq_filename, 'r') as f:
            data = json.load(f)
            for obj in data:
                tx = obj['tx']
                # print(tx)
                flattened = {
                    "tx_key": tx['key'],
                    "tx_sequence_number": tx['sequence_number'],
                    "tx_transaction_version": tx['transaction_version'],
                    "tx_amount": tx['data']['amount']['amount'],
                    "tx_currency": tx['data']['amount']['currency'],
                    "tx_metadata": tx['data']['metadata'],
                    "tx_receiver": tx['data']['receiver'],
                    "tx_sender": tx['data']['sender'],
                    "tx_type": tx['data']['type']
                }
                for k in obj.keys():
                    if not k == 'tx': 
                        flattened[k] = obj[k]
                
                # print(flattened)
                output.append(flattened)

print(output)

[{'tx_key': '00000000000000005f8ac83a9b3bf2eff20a6c16cd05c111', 'tx_sequence_number': 0, 'tx_transaction_version': 70167699, 'tx_amount': 500000000, 'tx_currency': 'GAS', 'tx_metadata': '6163636f756e742067656e65726174696f6e', 'tx_receiver': '5f8ac83a9b3bf2eff20a6c16cd05c111', 'tx_sender': '0cb4fef910b686e5ae39049fb892cb24', 'tx_type': 'receivedpayment', 'payment-round': 'PR20221001_001', 'artifacts': ['https://github.com/paulito123/ol-analytical-research'], 'contributor': '5F8AC83A9B3BF2EFF20A6C16CD05C111', 'contributor_role': 'broker', 'description': 'Coordinating the project, developing 50%', 'workgroup_id': 'analytics-wg', 'assignment_ref': 'https://airtable.com/appuiDshjiBzOAGl2/tblixhNzkEaKt3a75/viw77ncDURPkx2Mln/rec4D7I4k2KS4ytt5?blocks=hide', 'payment_type': 'work-related', 'reviewer': '5F8AC83A9B3BF2EFF20A6C16CD05C111'}, {'tx_key': '00000000000000005db97dd068263449c8eb369dc2312afa', 'tx_sequence_number': 104, 'tx_transaction_version': 70340591, 'tx_amount': 137853170, 'tx_curre

### Reorganise, enrich and filter report data

In [19]:
f = open(Config.CONTRIBUTORS_FILEPATH)
contributors = json.load(f)

artifacts = {}
data = []

for p_object in output:
    
    if "artifacts" in p_object:
        if len(p_object['artifacts']) > 0:
            artifacts[p_object['tx_key']] = p_object['artifacts']
        p_object.pop("artifacts")
    
    data.append(p_object)

df = pd.DataFrame(data, 
    columns = ['payment-round', 'workgroup_id', 'contributor', 'contributor_role',
               'description','payment_type','assignment_ref','reviewer',
               'tx_key', 'tx_sequence_number', 'tx_transaction_version',
               'tx_amount', 'tx_currency', 'tx_metadata', 'tx_receiver',
               'tx_sender', 'tx_type'])

print(df)  


    payment-round     workgroup_id                       contributor  \
0  PR20221001_001     analytics-wg  5F8AC83A9B3BF2EFF20A6C16CD05C111   
1  PR20221001_001     analytics-wg  5DB97DD068263449C8EB369DC2312AFA   
2  PR20221001_001     analytics-wg  E8DF1AFD258A10BD0392E1A8D274CC81   
3  PR20221010_001  hustle-karma-wg  5DB97DD068263449C8EB369DC2312AFA   
4  PR20221010_001     analytics-wg  E8DF1AFD258A10BD0392E1A8D274CC81   

           contributor_role                               description  \
0                    broker  Coordinating the project, developing 50%   
1                 developer                     This is another dummy   
2                                                 reviewing work done   
3  default-community-member                    documentation produced   
4                    tester                            testing of API   

   payment_type                                     assignment_ref  \
0  work-related  https://airtable.com/appuiDshjiBzOAGl2/tb